In [73]:
# https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py
# https://blog.csdn.net/weixin_43792166/article/details/97952312

import torch
from torch.autograd import Variable
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt


### Dataset

In [61]:
## load training and test data
train_data = datasets.MNIST(root='./mnist',
                            train = True,
                            transform=torchvision.transforms.ToTensor(),
                            download=True)

test_data = datasets.MNIST(root='./mnist',
                           train = False,
                           transform=torchvision.transforms.ToTensor(),
                           download=True)

In [62]:
## split training data into training and validation\
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(num_train * 0.25))
train_idx, valid_idx = indices[split:], indices[:split]
    

# define samplers for obtaining training and validation batches
from torch.utils.data.sampler import SubsetRandomSampler

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)    

In [63]:
## load data into batches
BATCH_SIZE = 64

train_loader = DataLoader(dataset=train_data, 
                          batch_size=BATCH_SIZE, 
                          sampler=train_sampler,
                          num_workers=0)

validation_loader = DataLoader(dataset=train_data, 
                               batch_size=BATCH_SIZE, 
                               sampler=valid_sampler,
                               num_workers=0)

test_loader = DataLoader(dataset=test_data, 
                         batch_size=BATCH_SIZE, 
                         num_workers=0)


### CNN model

In [64]:
## model

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # conv1
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        # conv2
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # flatten
        self.prediction = nn.Linear(32*7*7, 10)
        
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.prediction(x)
        return output

In [65]:
model = CNN()

In [66]:
## loss function
criterion = nn.CrossEntropyLoss()

## optimizer
optimizer = torch.optim.Adam(model.parameters())



In [67]:
## early stopping

from pytorch_lightning.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy',
                               min_delta=0.00,
                               patience=10,
                               verbose=False,
                               mode='max')


In [71]:
## train model

BATCH_SIZE = 64
N_EPOCHS = 100

train_losses = []
valid_losses = []
avg_train_losses = []  # average loss in each epoch
avg_valid_losses = []
early_stopping = EarlyStopping(patience=10, verbose=True)


for epoch in range(1, N_EPOCHS+1):
    ## train
    model.train()
    for batch, (batch_x, batch_y) in enumerate(train_loader, 1):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        optimizer.zero_grad()
        output = model(b_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    ## validate
    model.eval() 
    for batch_x, batch_y in validation_loader:
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        optimizer.zero_grad()
        output = model(b_x)
        loss = criterion(output, batch_y)
        valid_losses.append(loss.item())
    
    
    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)
    train_losses = []
    valid_losses = []
    
    
#     early_stopping(valid_losses, model)
#     if early_stopping.early_stop:
#         print("Early stopping")
#         break
        
        


EarlyStopping mode set to min for monitoring early_stop_on.


TypeError: 'EarlyStopping' object is not callable

In [75]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
import os

class LightningMNISTClassifier(pl.LightningModule):

  def __init__(self):
    super().__init__()

    # mnist images are (1, 28, 28) (channels, width, height) 
    self.layer_1 = torch.nn.Linear(28 * 28, 128)
    self.layer_2 = torch.nn.Linear(128, 256)
    self.layer_3 = torch.nn.Linear(256, 10)

  def forward(self, x):
      batch_size, channels, width, height = x.size()

      # (b, 1, 28, 28) -> (b, 1*28*28)
      x = x.view(batch_size, -1)

      # layer 1 (b, 1*28*28) -> (b, 128)
      x = self.layer_1(x)
      x = torch.relu(x)

      # layer 2 (b, 128) -> (b, 256)
      x = self.layer_2(x)
      x = torch.relu(x)

      # layer 3 (b, 256) -> (b, 10)
      x = self.layer_3(x)

      # probability distribution over labels
      x = torch.log_softmax(x, dim=1)

      return x

  def cross_entropy_loss(self, logits, labels):
    return F.nll_loss(logits, labels)

  def training_step(self, train_batch, batch_idx):
      x, y = train_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('train_loss', loss)
      return loss


  def validation_step(self, val_batch, batch_idx):
      x, y = val_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('val_loss', loss)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer


class MNISTDataModule(pl.LightningDataModule):

  def setup(self, stage):
    # transforms for images
    transform=transforms.Compose([transforms.ToTensor(), 
                                  transforms.Normalize((0.1307,), (0.3081,))])
      
    # prepare transforms standard to MNIST
    self.mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
    self.mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)

  def train_dataloader(self):
    return DataLoader(self.mnist_train, batch_size=64)

  def val_dataloader(self):
    return DataLoader(self.mnist_test, batch_size=64)

data_module = MNISTDataModule()

# train
model = LightningMNISTClassifier()
trainer = pl.Trainer()

trainer.fit(model, data_module)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


Extracting /Users/xieyanan/Desktop/DLproject/MNIST/raw/train-images-idx3-ubyte.gz to /Users/xieyanan/Desktop/DLproject/MNIST/raw


Extracting /Users/xieyanan/Desktop/DLproject/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/xieyanan/Desktop/DLproject/MNIST/raw


Extracting /Users/xieyanan/Desktop/DLproject/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/xieyanan/Desktop/DLproject/MNIST/raw


Extracting /Users/xieyanan/Desktop/DLproject/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/xieyanan/Desktop/DLproject/MNIST/raw
Processing...
Done!






  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33.0 K
2 | layer_3 | Linear | 2.6 K 
-----------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [76]:
train_loader

AttributeError: 'DataLoader' object has no attribute 'target'